In [2]:
#!/usr/local/bin/python
# -*- coding: utf-8 -*-
### export PYTHONIOENCODING=utf-8  # at cmd of linux
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.initializers import Constant
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras import regularizers
from keras import backend as K
from tensorflow.keras.callbacks import *
from tensorflow.keras import utils
import tensorflow as tf

import torch

Using TensorFlow backend.


In [3]:
import numpy as np
import pandas as pd 
import os
import matplotlib.pyplot as plt
import pickle
import time
import re
import nltk
import sys
import html
import xml.sax.saxutils as saxutils
from html.parser import HTMLParser
from io import StringIO
import random

from transformers import BertTokenizer, BertModel
from transformers import DistilBertModel,DistilBertTokenizer

from scipy.spatial.distance import cosine

from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
Stem=stemmer.stem

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
lemm=wordnet_lemmatizer.lemmatize

In [4]:
from gensim.models.fasttext import load_facebook_model

import fasttext.util
#fasttext.util.download_model('en', if_exists='ignore')  # English

In [5]:

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model_bert = DistilBertModel.from_pretrained('distilbert-base-uncased',
                                       output_hidden_states = True, # Whether the model returns all hidden-states.
                                       )  
'''
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = BertModel.from_pretrained('bert-base-uncased',
                                       output_hidden_states = True, # Whether the model returns all hidden-states.
                                       )
'''
print()

In [6]:
#print(len(cachedStopWords))
#print(len(cachedStopWords))
#print(type(cachedStopWords))

nltk_stopwords=set(cachedStopWords)

english_alghabet=['b','c','e','f','g','h','j','k','l','n','p','q','r','u','v','w','x','z']

numbers_remove=['one','two','three','four','five','six','seven','eight','nine','ten','tens','twenty',
                'fourty','fifty','sixty','seventy','eighty','ninety','hundred','hundreds','million','billion','trillion',
                'millions','thousand','thousands','second','third','forth','tenth','billions','trillions'] 

miscellaneous_remove=['absolutely', 'actually', 'adieu', 'ain', "ain't", 'aint', 'almost',
                       'awesome','awfully','amazing','interesting',
                       'alright','alrighty', 'amoungst', 'anybody', 'anymore', 'anyways', 'apart', 'apparently', 'anytime',
                       'appropriate',  'approximately', 'arent', 'behold', 'better', 'bravo','briefly','bad','best','brilliant',
                       'bye', 'cant', 'certainly', 'chrissakes', 'clearly', 'completely',
                       'congrat', 'congrats','congratulation', 'congratulations', 'consequently', 'cool', 'couldnt',
                       'darnit', 'de','dear', 'definitely','disappointing', 'didn', 'doesn', 'don', 'downwards',
                       'disgusting','dude','down','eg',"e.g.",'i.e.',
                       'encore','entirely', 'especially', 'et', 'etc', 'everybody', 'ex', 'exactly', 'excellent',
                       'fantastic','far', 'farewell','funny',
                       'felicitation', 'felicitations','finally', 'fully','furthermore', 'gadzooks', 
                       'good', 'goodby','goodness', 'gracious', 'great', 
                       'greetings', 'hallo', 'hardly', 'hasnt', 'haven', 'hello', 'here','hi', 'hither','higher','hopefully',
                       'here','there','including',
                       'howbeit', 'ie', 'immediately', 'inasmuch', 'inner', 'insofar', 'instead', 'inward', 'important',
                       'indeed','just', "it'd", "it'll", 'inside','kertyschoo', 'kg', 'km', 'lackaday', 
                       'largely', 'lately', 'later','lovely','large','big','small',
                       'lest', 'let', 'lets', 'likely', 'little', 'ltd', 'lower','magnificent', 'mainly', 'marvelous',
                       'myself','yourself','yourselves','himself','herself','hisself','ourselves','themsleves',
                       'maybe', 'meantime', 'merely', 'minus', 'near', 'nearly', 'necessary', 'never', 
                       'non', 'normally', 'obviously', 'ok', 'okay', 'ones', 'outside', 'over','other','others','only',
                       'overall', 'particular', 'particularly', 'please', 'plus', 'poorly', 'possible','up',
                       'possibly', 'potentially', 'predominantly', 'presumably', 'previously','primarily', 'probably',
                       'promising',
                       'promptly', 'readily', 'really', 'reasonably', 'recent', 'recently', 'ref',
                       'refs', 'regardless', 'related', 'relatively', 'respectively', 'resulting', 'right', 'sec', 
                       'secondly','self', 'selves', 'seriously', 'shall', 'shucks','somebody', 'somethan','sorry',
                       'somewhat', 'soon', 'late' , 'sorry', 'stupid', 'sub', 'substantially', 'successfully', 'sufficiently',
                       'useful',
                       'super', 'sure', "t's", 'th', 'thank', 'thanks', 'thanx', "that've", 'thats', 'there', "there'll",
                       "there've", 'thered', 'thereof', 'therere', 'theres', 'thereto', 'theyd', 'theyre', 'thorough',
                       'then','thankfully','too','today','yesterday','tomorrow','night',"morning",'afternoon','noon','tonight',
                       'evening','day','everyday', 'everynight','todays','nights','mornings','noons','afternoons','days',
                       'evenings','week','month','year',
                       'thoroughly', 'tnx', 'too','truly', 'twice', 'undoubtedly','unfortunately', 'unlike','unlikely',
                       'unto',  'usually', 'vs', 'welcome', 'well', 'went', 'werent', 'what', 'whatever', 'wheres', 'widely',
                       'wonderful', 'wont', 'wouldnt', 'wrong', 'worst','worse','www', 'yes', 'youd', 'youre', 'yummy', 
                       'zoinks','shit','literally','literal','pleasure','effective','fabulous','delighted',
                       'saturday','sunday','monday','tuesday','wednesday','thursday', 'friday','past','future','suitable',
                       'much','many','less','least','few','lots','lot','fewer','fewset','therefore','pm',
                       'afaik', 'br', 'idk','smh','qotd', 'ftw','bfn','yw', 'icymi','fomo','smdh', 'b4','imho',
                       'urdddd','fab' ,'delightful','absolute','pleasure','huge','latest','nowadays',
                       'january','february','april','june','july','august','september','october',
                       'november','december', 'autumn' ,'spring','winter','summer',
                       'mr','madam','sir','mrs','easy', 'difficult',
                       'weekend','south','north','west','east','asia','africa','europe','america','totally',
                       'come', 'comes', 'coming', 'came', 'seems', 'gives', 'gave', 'makes', 'made', 'keeps', 'kept', 
                       'calls', 'called', 'says', 'saying', 'said', 'goes', 'went', 'gone', 'got', 'saw', 'seen', 'shows',
                       'shown', 'took', 'taken', 'uses', 'moved', 'moves', 'puts',
                       'using','seem','give','make','keep','call','say','go','get','see','seems','seeming',
                       'seemed','show','take','made','used','move','become','became','becoming','becomes','put','use',
                       'find', 'finds', 'finding','aka',
                       'lol' , 'brb', 'lmk', 'ama', 'tbh', 'irl', "tl;dr", 'fml', 'bfn' ,' br', 'ht', "hth",'j/k', 'lmao' ] #cool

interjection_remove=['aaaahh', 'aaah', 'aaargh', 'aaay', 'aagh', 'aah',
                   'aargh', 'achoo', 'adios', 'ah', 'aha', 'ahem', 'ahh', 'ahhh',
                   'ahoy', 'alas', 'allo', 'amen', 'areet', 'argh', 'arrggh',
                   'arrividerci', 'asap', 'attaboy', 'avaunt', 'aw', 'aw', 'aww',
                   'awww', 'ay', 'ay', 'aye', 'ayeaugh', 'bada', 'badum', 'bah',
                   'bahaha', 'bam', 'bazinga', 'behold', 'bingce', 'bingo', 'blah',
                   'blech', 'bleh', 'blimey', 'bonjour', 'boo', 'booh', 'boohoo',
                   'booyah', 'bravo', 'brr', 'brrrr', 'btw', 'bwahaha', 'capeesh',
                   'capisce', 'cheerio', 'cheers', 'ciao', 'cor', 'cowabunga',
                   'crikey', 'cripes', 'da', 'dabba', 'dah', 'dammit', 'damn', 'dang',
                   'darn', 'de', 'dee', 'di', 'dizamn', 'doh', 'doo', 'drat', 'duh',
                   'dum', 'eeeek', 'eek', 'eep', 'egad', 'egads', 'eh', 'ehem', 'em',
                   'er', 'eureka', 'eww', 'ewww', 'eyh', 'fiddledeedee', 'fie',
                   'fore', 'foul', 'fuff', 'gah', 'gak', 'gee', 'geez', 'gesundheit',
                   'giddyap', 'golly', 'gosh', 'grr', 'grrrr', 'ha', 'hah', 'haha',
                   'hahaha', 'hallelujah', 'halloa', 'harrumph', 'harumph', 'haw',
                   'heck', 'heck', 'heeey', 'heh', 'hehe', 'hey', 'hhh', 'hic', 'hm',
                   'hmm', 'hmmm', 'hmmmm', 'hmmph', 'hmpf', 'ho', 'hola', 'hoo',
                   'hooray', 'howdy', 'hrmm', 'hrmph', 'hrmph', 'hrrmph', 'hu', 'huh',
                   'hullo', 'humph', 'hurrah', 'huzza', 'huzzah', 'ich', 'ick',
                   'ixnay', 'jeepers', 'jeez', 'kaboom', 'kapow', 'kerwham', 'la',
                   'lala', 'lo', 'lordy', 'meh', 'mhm', 'ml', 'mm', 'mmh', 'mmhm',
                   'mmm', 'muahaha', 'mwah', 'mwahaha', 'na','nay','nah', 'nanu', 'nooo', 'nope',
                   'nuh', 'oh', 'ohh', 'oho', 'oi', 'okeydoke', 'om', 'oof', 'ooh',
                   'oomph', 'oooh', 'ooooh', 'oops', 'ouch', 'ow', 'oww', 'oy',
                   'oyez', 'oyh', 'pew', 'pff', 'pffh', 'pfft', 'phew', 'phut',
                   'phweep', 'phwoar', 'phwoarr', 'poof', 'poogh', 'prethee',
                   'prithee', 'prosit', 'pssh', 'psst', 'queep', 'roger', 'salaam',
                   'salam', 'sheesh', 'shh', 'shhh', 'shitfire', 'shoo', 'shoop',
                   'shush', 'sigh', 'sssh', 'strewth', 'ta', 'tarnations', 'tchah',
                   'teehee', 'tish', 'touché', 'tsk', 'tss', 'tut', 'uggh', 'ugh',
                   'uh', 'uhh', 'uhm', 'um', 'umm', 'ummm', 'umph', 'unh', 'upadaisy',
                   'upsadaisy', 'ur', 'urgh', 'vay', 'vayf', 'viva', 'voila', 'waa',
                   'waaaaah', 'waah', 'wah', 'wahey', 'wassup', 'weee', 'welp',
                   'wham', 'whamo', 'whee', 'whew', 'whizz', 'whoa',
                   'whoo', 'whoopee','whoop', 'whoops', 'whoopsy', 'whoosh', 'woah', 'woo',
                   'woohoo', 'wotcha', 'wotcher', 'wow', 'wowsers', 'wowsers',
                   'wuzzup', 'wuzzup', 'wuzzup', 'ya', 'yabba', 'yada', 'yadda',
                   'yak', 'yarooh', 'yay', 'yea', 'yeah', 'yech', 'yee', 'yeeeeaah',
                   'yeehaw', 'yeow', 'yes', 'yessiree', 'yew', 'yikes', 'yippee',
                   'yo', 'yoo', 'yoohoo', 'yow', 'yowza', 'yuck', 'yuh', 'zing',
                   'zoiks', 'zomfg', 'zomg', 'zounds', 'zut']
             
import spacy
sp = spacy.load('en_core_web_sm')
spacy_stopwords = sp.Defaults.stop_words
type(spacy_stopwords)
#spacy_exclude=['using','name','seem','give','make','keep','call','say','go','get','see','seems','seeming',
#               'seemed','show','take','made','used','move','become','became','becoming','becomes','put','use']# serious

from stop_words import get_stop_words
stop_words = get_stop_words('en')
stop_words1 = get_stop_words('english')
#print(type(stop_words1))
#print()
#print(stop_words1)
lib_stopwords=set(stop_words1)

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
#print(type(ENGLISH_STOP_WORDS))
#print()
#print(set(ENGLISH_STOP_WORDS))
#sklearn_exclude=['find','get','found','go','see','seem','seems','give','seemed','take','keep','show','put','made'] # system  cry
sklearn_stopwords=set(ENGLISH_STOP_WORDS)

#spacy_stopwords.difference_update(set(spacy_exclude))
#sklearn_stopwords.difference_update(set(sklearn_stopwords))
#for removing "just" one item, use "remove"
temp_1=set([])
#temp_1.update(nltk_stopwords)
#temp_1.update(lib_stopwords)
#temp_1.update(sklearn_stopwords)
#temp_1.update(spacy_stopwords)
#temp_1.update(set(english_alghabet))
#temp_1.update(set(numbers_remove)) 
#temp_1.update(set(miscellaneous_remove))
#temp_1.update(set(interjection_remove))
#temp_1.update(['rt','be','will','was','were','is','am','are','have','has','had','do','does','done'])
#temp_1.update(['rt'])
cachedStopWords=temp_1
#len(cachedStopWords)

In [7]:
class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [8]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

#print(lemmatizer.lemmatize("I am loving it")) #I am loving it
#print(lemmatizer.lemmatize("loving")) #loving
#print(lemmatizer.lemmatize("loving", "v")) #love
#print(lemmatize_sentence("I am loving it")) #I be love it

In [9]:
def cleaning (text):
    
    #order of lines is important
    
    text=strip_tags(text)
    #text=html.unescape(text)   # stripping or converting html entities 
    #text=saxutils.unescape(text) 
    
    #convertings words that their lower and uper cases are different
    text=re.sub(" US | U\.S\. ", ' USA ', text) # before lower
    
    #converting
    text = re.sub("“|”", ''' " ''', text)  #before next lines
    text = re.sub("’|′|‘|`", " ' ", text)  #before next lines
    
    #removing tabs and lines
    text=re.sub('\t|\n', ' ', text)
    
    #converting lower_case
    text = text.lower() 
    
    #converting
#    text=re.sub('\$|£|€|¥|dollar|dollars|yen|yens|euros', ' money ', text)   # not euro 
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    
    #removing emoji
    text = emoji_pattern.sub(r' ', text) 

    #removing emojis and non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]+|,Ä¶',' ', text)  

    #removeing http and https (URL)
    text = re.sub(r'(http://|https://)\S+', '', text)
    
    #removing www (URL)
    text=re.sub(r'www\.\S+', '', text)
    
    #removing targets
    text=re.sub('( |^)@\S+', '', text) 

    '''
    #removing common expressions
    text=re.sub("looking forward to|look forward to|make sure|kidding me|\
                |in my opinion|by the way,|as soon as possible|shaking my head|i don't know|I do not know|\
                |in real life|quote of the day|as far as i know|shake my head|\
                |to be honest|in other words|let me know|just kidding|hope that helps|hat tip|\
                |just like that|happy birthday|never mind|well-done|\
                |in my humble opinion|happy new year|you're welcome|you are welcome| \
                |it doesn't matter|it does not matter|i think|i wonder|do you think", ' ', text)  
    '''
    
    #convertings
    text=re.sub("can't", 'cannot', text) # before other n't 
    text=re.sub("can not ", 'cannot ', text)  
    text=re.sub("'ve",' have', text)
    text=re.sub("n't",' not', text)
    text=re.sub("'ll",' will', text)
#    text=re.sub("'d",' would', text)
    text=re.sub("'re",' are', text)
    text=re.sub("i'm",'i am', text)
    text=re.sub("&",' and ', text)
    text=re.sub(" w/ ",' with ', text)
    text=re.sub(" w/i | w/in ",' within ', text)
    text=re.sub(" w/o ",' without ', text)
    text=re.sub(" c/o ",' care of ', text)
    text=re.sub(" h/t ",' hat tip ', text)
    text=re.sub(" b/c ",' because ', text)
#    text=re.sub("=",' equals to ', text)
    text=re.sub("=",' = ', text)
#    text=re.sub("\+",' plus ', text)
    text=re.sub("\+",' + ', text)
    text=re.sub("united states",'usa', text)
    text=re.sub("united kingdom",'uk', text)
    text=re.sub(" the us ",' usa ', text)
    text=re.sub("start-up|start_up",'startup', text)
    text=re.sub("u\.s\.a", 'usa', text)  #try text=re.sub("u.s.a", 'usa', text) with text=substantially 
    #text=re.sub("aka", 'also known as', text)     
    text=re.sub("'"," ' ", text)     
    
    text= re.sub("(\?)+", '? ',text)     
    text= re.sub("(!)+", '! ',text)     
    text= re.sub("(\.\.)+", ' ',text)   

#    text = "".join(lemmatize_sentence(text))
    
    #removing some special charachter  
#    text= re.sub("[\"\+\-\|\*\?\(\)\/\\\^\[\]``<>\.{}`′’‘'_;•«»,@:~!\=%&]+", ' ',text) 
#    text= re.sub("[\"\“\”\+\-\|\*\?\(\)\/\\\^\[\]\.{}_`′’‘';•«,@:~!\=%&]+", ' ',text) 
    
    #removing hashtag
#    text=re.sub('#', ' ', text) 
    
    #removing numbers not attached to alphabets
    '''
    text=re.sub("(^)(\d+)?(\.)?(\d+)? ",' ',text)   #removing numer at the beginning
    text=re.sub("(\s)[0-9]?(\.)?(\d+) ",' ',text) #py6 and py9
    text= re.sub(" (\.)(\d+) ", ' ',text)
    text= re.sub(" (\d+) (\d+) (\d+) (\d+) (\d+) ", ' ',text)
    text= re.sub(" (\d+) (\d+) (\d+) (\d+) ", ' ',text)
    text= re.sub(" (\d+) (\d+) (\d+) ", ' ',text)
    text= re.sub(" (\d+) (\d+) ", ' ',text)
    text= re.sub(" (\d+) ", ' ',text)
    text= re.sub(" (\d+)$", ' ',text)
    '''
    #text=re.sub("\S+(\d+) ",' ',text) # alphabet+digit (attached)
    #text=re.sub(" (\d+)\S+",' ',text) # digit+alphabet (attached)
    #text=re.sub(" \S+(\d+)\S+ ",' ',text) # alphabet+digit+alphabet (attached)
    #text=re.sub("(\d+)",' ',text)  #removing any number anywhere but keeps \. for decimal numbers

    #removing space
    text=re.sub('\s+',' ',text)    
    
    text=re.sub('(^)rt ','',text)    # if we do not want to remove stopwords

#    text= nltk.word_tokenize(text) # necessary for removing stopwords
    #text= text.split() #sometimes

    #removing_stopwords 
    #text_without_sw = [word.lower() for word in text if word.lower() not in stopwords.words()] #very slow
#    text = [word for word in text if word not in cachedStopWords]

    #lemmatization
    #text= [ lemm(word, pos="v") for word in text]
    #text= [ lemm(word, pos="n") for word in text]
    #text= [ lemm(word, pos="a") for word in text]
    
    #stemming 
    #text = [Stem(word) for word in text]
    
#    text=' '.join(text)
#    text=re.sub("''",'''"''', text)    #since nltk.tokenize converts second " to ''"
#    text=re.sub("``",'''"''', text)   # since nltk.tokenize converts first " to " ``
    
    return text

In [10]:
'''
import psycopg2
con = psycopg2.connect(database="postgres", user="postgres", password="Jafarsql", host="localhost", port="5432")
print("Database opened successfully")

cur = con.cursor()
##cur.execute("SELECT user_id, tweet from ent_2019_100K limit 100000 ")
cur.execute("SELECT user_id, tweet from ent_2019_1000k ")
rows_ent = cur.fetchall()
con.close()

print(len(rows_ent))
'''
pass

In [11]:
'''
#df_ent = pd.read_csv('/archives1/Datasets/TweetsWorld/ent_tweets_world.csv', delimiter='\t', na_values=".",error_bad_lines=False)#,warn_bad_lines=False)
df_ent = pd.read_csv('ent_tweets_world.csv', delimiter='\t', na_values=".",error_bad_lines=False)#,warn_bad_lines=False)
print(df_ent.shape)
print(df_ent.columns)
#print(df_ent.head()) # Preview the first 5 lines of the loaded data 

rows_ent=list(df_ent[['user_id', 'tweet','tweet_created_at']].itertuples(index=False, name=None)) #rows_ent0
#rows_ent= list(zip(df_ent.user_id, df_ent.tweet))
#rows_ent=df_ent[['user_id','tweet']].apply(tuple, axis=1) 
del df_ent

print("Number of tweets in ent:",len(rows_ent))  #rows_ent0
print('Memory size of ent:',sys.getsizeof(rows_ent)) #rows_ent0
'''
pass

In [12]:
year='2017'

In [13]:
#with open("ent_filtering.txt", "rb") as fp:   
#    ent_filter = pickle.load(fp)

In [14]:
#with open("./years/"+year+"_rand_2M_e.txt", "rb") as fp:   
#    rand_2M_e=pickle.load(fp )

In [15]:
#with open("./years/"+year+"_seed_ent_pos_1.txt", "rb") as fp:   
#    id_ent_rel = pickle.load(fp)

In [16]:
#with open("./years/"+year+"_seed_ent_neg_1.txt", "rb") as fp:   
#    id_ent_irrel = pickle.load(fp)

In [17]:
#with open("./years/"+year+"_tweets_ent_scores_1.txt", "rb") as fp:   
#    dic_ent_tweets_scores = pickle.load(fp)

In [18]:
#len(id_ent_rel), len(id_ent_irrel), len(id_ent_rel)+len(id_ent_irrel)

In [19]:
with open("./years/"+year+"_ent_year_ind.txt", "rb") as fp:   
    ent_year_ind=pickle.load(fp)

In [20]:
len(ent_year_ind)

2000000

In [21]:
'''
t0 = time.time()

ent_year_ind_all=set()
#ent_tweets_rows=[]

for k, i in enumerate(rows_ent):
    #ent_tweets_rows.append(cleaning(i[1]))
    if year in i[2]:
        if str(i[0]) not in ent_filter:
            ent_year_ind_all.add(k)

print( time.time() - t0)

len(ent_year_ind_all)
'''
pass

In [22]:
with open("./years/"+year+"_ent_year_ind.txt", "rb") as fp:   
    ent_year_ind=pickle.load(fp)

len(ent_year_ind)

2000000

In [23]:
'''
t0 = time.time()

#ent_users_rows=[]
ent_tweets_rows=[]
for i in rows_ent:
    #ent_users_rows.append(i[0])
    ent_tweets_rows.append(cleaning(i[1]))
    
print( time.time() - t0)

#ent_users_rows_np=np.array(ent_users_rows)  
#ent_users=np.unique(ent_users_rows_np)
#print(len(ent_users))
'''
pass

In [24]:
#with open("./years/ent_cleaning_classification.txt", "wb") as fp:  
#    pickle.dump(ent_tweets_rows, fp , protocol=4)

In [25]:
with open("./years/ent_cleaning_classification.txt", "rb") as fp:  
    ent_tweets_rows=pickle.load(fp)

In [26]:
'''
t0 = time.time()

mng_users_rows=[]
mng_tweets_rows=[]
for i in rows_mng:
    mng_users_rows.append(i[0])
    mng_tweets_rows.append(cleaning(i[1]))

print( time.time() - t0)

mng_users_rows_np=np.array(mng_users_rows)  
mng_users=np.unique(mng_users_rows_np)
print(len(mng_users))
'''
pass

In [27]:
def embedder (tweets):
    # padding after {SEP}
    embedding_dim=768
    t0 = time.time()
    max_len_tokens=20
    num_tweets=np.shape(tweets)[0]
    tweets_embedded= np.zeros([num_tweets,max_len_tokens,embedding_dim],dtype="float32")
    #mng_full_embedding_word=[]
#    mng_full_tokenized=[]
    
    for i, text in enumerate(tweets):

        marked_text = "[CLS] " + text + " [SEP]"
        tokenized_text = tokenizer.tokenize(marked_text)
#        print('a1',tokenized_text)
        
        # truncate if size of tekens is more than max_len_tokens
        if len(tokenized_text)>max_len_tokens+2:
#            print('a2',max_len_tokens+2)
            tokenized_text= tokenized_text[0:max_len_tokens+2]
#            print('a3',tokenized_text)
            tokenized_text[max_len_tokens+2-1]="[SEP]"
#            print('a4',tokenized_text)

        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
#        print('a5',indexed_tokens)
        indexed_tokens = indexed_tokens + [0] * (max_len_tokens+2 - len(indexed_tokens))
#        print('a6',indexed_tokens)

        segments_ids = [1] * len(tokenized_text)
        temp=len(tokenized_text)
        segments_ids= segments_ids + [0] * (max_len_tokens+2 - len(segments_ids)) 
#        print('a7',segments_ids) 

        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])
        with torch.no_grad():
            outputs = model_bert(tokens_tensor, segments_tensors)
            out = outputs[0]

        token_embeddings = torch.squeeze(out, dim=0)  
#        print('a8',token_embeddings)
        token_embeddings=np.delete(token_embeddings, (0,temp-1), axis = 0)

        tweets_embedded[i]=token_embeddings
#        mng_full_tokenized.append(tokenized_text)

#    print(np.shape(tweets_embedded))
#    print( time.time() - t0)
    return tweets_embedded    

In [28]:
def embedder2 (tweets):
    #padding before {SEP}
    embedding_dim=768
    t0 = time.time()
    max_len_tokens=20
    num_tweets=np.shape(tweets)[0]
    tweets_embedded= np.zeros([num_tweets,max_len_tokens,embedding_dim],dtype="float32")
    #mng_full_embedding_word=[]
#    mng_full_tokenized=[]
    
    for i, text in enumerate(tweets):

        marked_text = '[CLS]'+ text 
        tokenized_text = tokenizer.tokenize(marked_text)
#        print('a1',tokenized_text)
        temp=len(tokenized_text)
#        print(temp)
        # truncate if size of tekens is more than max_len_tokens
        if temp>=max_len_tokens+2:
#            print('a2',max_len_tokens+2)
            tokenized_text= tokenized_text[0:max_len_tokens+2]
#            print('a3',tokenized_text)
            tokenized_text[max_len_tokens+2-1]="[SEP]"
#            print('a4',tokenized_text)
        else:
#            print(type(tokenized_text))
            tokenized_text = tokenized_text + ['[PAD]'] * (max_len_tokens+1 - temp)+ ['[SEP]']
#            print('a44',tokenized_text)

#            tf.keras.preprocessing.sequence.pad_sequences(
#                sequences, maxlen=10, dtype='int32', padding='post',
#                truncating='post', value=0.0)

        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
#        print('a5',indexed_tokens)
#        indexed_tokens = indexed_tokens + [0] * (max_len_tokens+2 - len(indexed_tokens))
#        print('a6',indexed_tokens)

#        segments_ids = [1] * len(tokenized_text)
        segments_ids= [1]* min(temp,max_len_tokens+1) + [0] * (max_len_tokens+1 - temp)  + [1]
#        print('a7',segments_ids) 

        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])
        with torch.no_grad():
            outputs = model_bert(tokens_tensor, segments_tensors)
            out = outputs[0]

        token_embeddings = torch.squeeze(out, dim=0)  
#        print('a8',token_embeddings)
        token_embeddings=np.delete(token_embeddings, (0,-1), axis = 0)

        tweets_embedded[i]=token_embeddings
#        mng_full_tokenized.append(tokenized_text)

#    print(np.shape(tweets_embedded))
#    print( time.time() - t0)
    return tweets_embedded    

In [29]:
def embedder0 (tweets):
    #padding before {SEP} forcing zero
    embedding_dim=768
    t0 = time.time()
    max_len_tokens=30
    num_tweets=np.shape(tweets)[0]
    tweets_embedded= np.zeros([num_tweets,max_len_tokens,embedding_dim],dtype="float32")
#    print('a11',np.shape(vector_temp))

    #mng_full_embedding_word=[]
#    mng_full_tokenized=[]
    
    for i, text in enumerate(tweets):
        
        
                
        marked_text = '[CLS]'+ text + '[SEP]'
        tokenized_text = tokenizer.tokenize(marked_text)
        segments_ids = [1] * len(tokenized_text)
#        print('a1',tokenized_text)
        temp=len(tokenized_text)-2
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])
        with torch.no_grad():
            outputs = model_bert(tokens_tensor, segments_tensors)
            out = outputs[0]
        token_embeddings = torch.squeeze(out, dim=0)  
#        print('a8',token_embeddings)
        token_embeddings=np.delete(token_embeddings, (0,-1), axis = 0)
#        print('a88',len(token_embeddings))
        if len(token_embeddings)>=max_len_tokens:
            vector_temp= token_embeddings[0:max_len_tokens]
#            print('a9',vector_temp)
        else:
            vector_temp = np.zeros([max_len_tokens,embedding_dim],dtype="float32")
            vector_temp[0:temp] = token_embeddings
#            print('a99',vector_temp)
        tweets_embedded[i]=vector_temp
#        mng_full_tokenized.append(tokenized_text)

#    print(np.shape(tweets_embedded))
#    print( time.time() - t0)
    return tweets_embedded    

In [30]:
#aa=embedder0(['That is very good and joy '])
#aa.shape

In [31]:
#rand_inds1e=random.sample(range(0, 1000000), 300000) #200000

In [32]:
#with open("rand_inds1e.txt", "wb") as fp:   
#    pickle.dump(rand_inds1e, fp , protocol=4)

In [33]:
'''
temp_set=set()
seed_public0=[]
for i in rand_inds1e:
    temp=public_tweets_rows[i]
    if temp !='' and temp !=' ':
        if temp not in temp_set:
            seed_public0.append(public_tweets_rows[i])
            temp_set.add(temp) # in order that we do not have repetative tweets
'''
pass

In [34]:
'''
# be careful: it has been changed 
seed_negative_all=[]
for index in id_ent_irrel:
    #print(index)
    #print(ent_tweets_rows[index])
    #print(rows_ent[index])
    seed_negative_all.append(ent_tweets_rows[index])  

len(seed_negative_all)
'''
pass

In [35]:
'''
seed_ent_all=[]
for index in id_ent_rel:
    #print(index)
    #print(ent_tweets_rows[index])
    #print(rows_ent[index]) 
    seed_ent_all.append(ent_tweets_rows[index])    

len(seed_ent_all)
'''
pass

In [36]:
#indices=np.arange(len(x_data))

In [37]:
'''
train_texts, test_texts, y_train, y_test,idx1,idx2 = train_test_split(x_data, y_data, indices, test_size=.2, shuffle=True, stratify= y_data)
print("train set size " + str(len(train_texts)))
print("test set size " + str(len(test_texts)))
'''
pass

In [38]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
#tknzr.tokenize(text)

In [39]:
'''
t0=time.time()
train_texts_tokens=[]
for text in train_texts:
#    tokens=text.split()
    #tokens=nltk.word_tokenize(text)
    tokens=tknzr.tokenize(text)
    train_texts_tokens.append(tokens)
    
print(len(train_texts_tokens))
time.time()-t0
'''
pass

In [40]:
ft = fasttext.load_model('cc.en.300.bin')
print(ft.get_dimension())

300


In [41]:
#fasttext.util.reduce_model(ft, 100)
#ft.get_dimension()

In [42]:
'''
t0=time.time()
embedding_dim=300
max_tokens=50
#X_train_test=[]
x_train=np.zeros([len(train_texts_tokens),max_tokens,embedding_dim],dtype='float32')

for kk,tokens in enumerate(train_texts_tokens):
    if tokens !="" and tokens!=' ' and  tokens!=[]:   
        a1=[]
        for token in tokens:
            a1.append( ft.get_word_vector(token) )
                    
        temp=np.zeros([max_tokens,embedding_dim])
        if len(tokens)>max_tokens:
            temp=a1[0:max_tokens]
        elif len(tokens)==max_tokens:
            temp=a1
        else: # if len(tokens)<max_tokens:
            temp[0:len(tokens)]=a1
#        X_train_test.append(temp) 
        x_train[kk]=temp

    else:
        temp=np.zeros([max_tokens,embedding_dim])
#        X_train_test.append(temp)
        x_train[kk]=temp
        
time.time()-t0
'''
pass

In [43]:
#x_train = tf.cast(x_train, tf.float32)
#y_train = tf.cast(y_train, tf.float32)

In [44]:
#x_train1, x_test1, y_train, y_test = train_test_split(data, y_data, test_size=0.2, shuffle=True, stratify= y_data)   

In [45]:
sequence_length=50
num_filters=100
embedding_dim=300   #768 for bert
dropout_rate=0.2

In [46]:
'''
inputs = Input(shape=(sequence_length,embedding_dim,), dtype='float32')
#inputs = Input(shape=(sequence_length,), dtype='float32')

#embedded_inputs = embedding_layer(inputs)
#embedding_layer = Embedding(input_dim=20000, output_dim=embedding_dim, input_length=sequence_length, weights=[inputs])(inputs)

#inputs_reshaped = Reshape((sequence_length, embedding_dim, 1))(embedded_inputs)
inputs_reshaped = Reshape((sequence_length, embedding_dim, 1))(inputs)

conv_1 = Conv2D(num_filters, kernel_size=(1, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(inputs_reshaped)
#conv_1 = LeakyReLU(alpha=0.2)(conv_1) #without activation at Conv2D
conv_1 = BatchNormalization()(conv_1)

conv_2 = Conv2D(num_filters, kernel_size=(2, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(inputs_reshaped)
#conv_2 = LeakyReLU(alpha=0.2)(conv_2)
conv_2 = BatchNormalization()(conv_2)

conv_3 = Conv2D(num_filters, kernel_size=(3, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(inputs_reshaped)
#conv_3 = LeakyReLU(alpha=0.2)(conv_3)
conv_3 = BatchNormalization()(conv_3)

conv_4 = Conv2D(num_filters, kernel_size=(4, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(inputs_reshaped)
#conv_4 = LeakyReLU(alpha=0.2)(conv_4)
conv_4 = BatchNormalization()(conv_4)

conv_5 = Conv2D(num_filters, kernel_size=(5, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(inputs_reshaped)
#conv_5 = LeakyReLU(alpha=0.2)(conv_5)
conv_5 = BatchNormalization()(conv_5)

conv_6 = Conv2D(num_filters, kernel_size=(6, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(inputs_reshaped)
#conv_6 = LeakyReLU(alpha=0.2)(conv_6)
conv_6 = BatchNormalization()(conv_6)

conv_7 = Conv2D(num_filters, kernel_size=(7, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(inputs_reshaped)
#conv_7 = LeakyReLU(alpha=0.2)(conv_7)
conv_7 = BatchNormalization()(conv_7)

conv_8 = Conv2D(num_filters, kernel_size=(8, embedding_dim), activation='relu', kernel_regularizer=regularizers.l2(3))(inputs_reshaped)
#conv_8 = LeakyReLU(alpha=0.2)(conv_8)
#conv_8 = BatchNormalization()(conv_8)

maxpool_1 = MaxPool2D(pool_size=(sequence_length - 1 + 1, 1), strides=(1,1))(conv_1)
maxpool_2 = MaxPool2D(pool_size=(sequence_length - 2 + 1, 1), strides=(1,1))(conv_2)
maxpool_3 = MaxPool2D(pool_size=(sequence_length - 3 + 1, 1), strides=(1,1))(conv_3)
maxpool_4 = MaxPool2D(pool_size=(sequence_length - 4 + 1, 1), strides=(1,1))(conv_4)
maxpool_5 = MaxPool2D(pool_size=(sequence_length - 5 + 1, 1), strides=(1,1))(conv_5)
maxpool_6 = MaxPool2D(pool_size=(sequence_length - 6 + 1, 1), strides=(1,1))(conv_6)
maxpool_7 = MaxPool2D(pool_size=(sequence_length - 7 + 1, 1), strides=(1,1))(conv_7)
maxpool_8 = MaxPool2D(pool_size=(sequence_length - 8 + 1, 1), strides=(1,1))(conv_8)

#concatenated_e1 = Concatenate(axis=3)([maxpool_1, maxpool_2, maxpool_3, maxpool_4, maxpool_5])
concatenated_e1 = Concatenate(axis=3)([ maxpool_1, maxpool_2, maxpool_3, maxpool_4, maxpool_5, maxpool_6, maxpool_7 ])

X = Flatten()(concatenated_e1)

X = Dense(units=512, activation='linear')(X) 
X = Dropout(dropout_rate)(X)

X = Dense(units=256, activation='linear')(X) 
X = Dropout(dropout_rate)(X)

X = Dense(units=128, activation='linear')(X) 
X = Dropout(dropout_rate)(X)

X = Dense(units=64, activation='linear')(X) 
X = Dropout(dropout_rate)(X)

X = Dense(units=32, activation='linear')(X) 
X = Dropout(dropout_rate)(X)

X = Dense(units=16, activation='linear')(X)
X = Dropout(dropout_rate)(X)

X = Dense(units=8, activation='linear')(X)
X = Dropout(dropout_rate)(X)

#outputs = Dense(units=5, activation='softmax')(X) 
outputs = Dense(units=1, activation='sigmoid')(X) 

classifier = keras.Model(inputs, outputs)
classifier.summary()
#classifier.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3) , loss= 'categorical_crossentropy', metrics=['accuracy'])
classifier.compile(optimizer=keras.optimizers.Adamax() , loss= 'binary_crossentropy', metrics=['accuracy'])

#optim= adamax, adam
#loss:categorical_crossentropy, KLDivergence,
'''
pass

In [47]:
'''
# not good results ##type 2( small kernels like images)
#sequence_length=16 # or 24 or 32 or 64 
##encoder
print(sequence_length,embedding_dim)
inputs = Input(shape=(sequence_length,embedding_dim,), dtype='float32')

x = Reshape((sequence_length, embedding_dim, 1))(inputs)
print(x.shape)
x = Conv2D(8, (4,4), activation='relu', padding='same')(x)
print(x.shape)
x = MaxPooling2D((2, 3), padding='same')(x)
print(x.shape)
x = Conv2D(8, (4,4), activation='relu', padding='same')(x)
print(x.shape)
x = MaxPooling2D((2, 3), padding='same')(x)
print(x.shape)
x = Conv2D(8, (2, 2), activation='relu', padding='same')(x)
print(x.shape)
x = MaxPooling2D((2, 3), padding='same')(x) 
print(x.shape)

#print(x.shape)

x = Flatten()(x)
print(np.shape(x))
#x = Dense(1764, activation="relu")(x)

#x = Dense(units=8, activation='linear')(x)
#x = Dropout(dropout_rate)(x)

outputs = Dense(units=5, activation='softmax')(x) 

classifier = keras.Model(inputs, outputs)
#optim = keras.optimizers.Adam(learning_rate=0.0001)
#classifier.compile(optimizer=optim)
classifier.compile(optimizer=keras.optimizers.Adam() , loss= 'categorical_crossentropy', metrics=['accuracy'])
classifier.summary()
'''
pass

In [48]:
'''
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-6, patience=5, verbose=1, mode='auto', restore_best_weights=True)

#history= classifier.fit(x_train,x_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=1,batch_size=32,epochs=1000)
print(np.shape(x_train))

history=classifier.fit(x_train,y_train,
                        epochs=10,
                        #callbacks=[monitor],
                        batch_size=512, # or 64
                        shuffle= True,
                        verbose=1,
                        validation_split=0.125) #0.25

#history = classifier.fit(x_train, y_train, epochs=8, batch_size=512, verbose=1, validation_split=0.25, shuffle=True)
'''
pass

In [49]:
'''
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
'''
pass

In [50]:
#classifier.save('./all_neg_first_pos/2018/case2')

In [51]:
classifier = keras.models.load_model('./all_neg_first_pos/2018/case3')

In [52]:
'''
# for bert
t0=time.time()
ent_ind_detected=[]
b=ent_tweets_rows[0:200000]
ent_test_embedded=embedder0(b)
y_predicted=classifier.predict(ent_test_embedded)
for ind, value in enumerate(y_predicted):
    if value >= 0.5:
        ent_ind_detected.append(ind)
print(time.time()-t0) 
'''
pass

In [53]:
time.sleep(1*60)

In [54]:
'''
t0=time.time()

max_tokens=50
embedding_dim=300
batch_size=1000
batch=0
#len_ent=len(ent_tweets_rows)
len_ent=len(test_texts)

results_ent=np.zeros([len_ent,1])

while batch <len_ent:
    test =  test_texts [batch : batch + batch_size]

#    test =  ent_tweets_rows [batch : batch + batch_size]
    test_data=[]
    
    for tweet in test:
    #    tokens=text.split()
        #tokens=nltk.word_tokenize(tweet)
        tokens=tknzr.tokenize(tweet)
        if tokens !="" and tokens!=' ' and  tokens!=[]:   
            a1=[]
            for token in tokens[0:max_tokens]:
                a1.append( ft.get_word_vector(token) )
            
            a1=np.asarray(a1, dtype=np.float32)            
            temp=np.zeros([max_tokens,embedding_dim])

            if len(tokens)>max_tokens:
                temp=a1[0:max_tokens]
            elif len(tokens)==max_tokens:
                temp=a1
            else: #if len(tokens)<max_tokens:
                temp[0:len(tokens)]=a1
        
            #x_test=temp.reshape((1,max_tokens,embedding_dim))  
            #print(np.shape(temp))
            #x_test = tf.cast(temp, tf.float32)
            test_data.append(temp)
            
        else:
            test_data.append(np.zeros([max_tokens,embedding_dim]))
              
#    test_data = tf.cast(test_data, tf.float32)
    test_data = np.asarray(test_data, dtype=np.float32)
    res_test=classifier.predict(test_data)
    results_ent [ batch: batch +len (res_test)]= res_test
    batch += batch_size
     
time.time()-t0
'''
pass

In [55]:
#len(results_ent),len(y_test)

In [56]:
'''
n_correct=0
false_val=[]
for k,i in enumerate(results_ent):
    #print(k)
    #print(i)
    if i>0.5:
        temp=1
    else:
        temp=0
    
    if temp == y_test[k]:
        n_correct +=1
    else:
        false_val.append(i)
        
        
accuracy=    n_correct/len(results_ent)
accuracy
'''
pass

In [57]:
#new_test=seed_ent_all[0:len(seed_ent_all)-len(seed_negative)]
#len(new_test)

In [58]:
#with open("./years/"+year+"_tweets_ent_scores_1.txt", "rb") as fp:  
#    dic_ent_tweets_scores=pickle.load(fp)

In [59]:
#id_unused=id_ent_rel+id_ent_irrel

In [60]:
#id_unused=id_ent_rel[num:]+id_ent_irrel[num:]

In [61]:
'''
rand_inds=np.sort(random.sample(range(0, len(id_unused)), 100000))
len(rand_inds)
'''
pass

In [62]:
'''
id_new=[]
for i in rand_inds:
    id_new.append(id_unused[i])
'''
pass

In [63]:
#id_new=id_unused
#len(id_new)

In [64]:
'''
for i in id_new[0:0]:
    print(i)
    if i in id_unused:
        print('yes')
'''
pass

In [65]:
new_test=[]
for i in ent_year_ind:
    new_test.append(ent_tweets_rows[i])

In [66]:
len(new_test)

2000000

In [67]:
#new_test=seed_ent_all[len(seed_negative):]
#len(new_test)

In [68]:
#new_test1=seed_ent_all[len(seed_ent):]
#len(new_test1)

In [69]:
#new_test2=seed_negative_all[len(seed_negative):]
#len(new_test2)

In [70]:
#new_test=new_test1+new_test2

In [71]:
#indices_unused=id_ent_rel[num:]+id_ent_irrel[num:]

In [72]:
#dic_ent_tweets_scores[id_ent_rel[num]]

In [73]:
#dic_ent_tweets_scores[id_ent_irrel[num]]

In [74]:
#scores=list(dic_ent_tweets_scores.items())

In [75]:
'''
for i , k in scores[0:5]:
    print(i,k)
    
print()

for i , k in scores[-5:-1]:
    print(i,k)
'''
pass

In [79]:
t0=time.time()

embedding_dim=300
max_tokens=50
batch_size=1000
batch=0

len_ent_test_new=len(new_test)

results_ent_test_new=np.zeros([len_ent_test_new,1])

while batch <len_ent_test_new:
    test =  new_test [batch : batch + batch_size]
    test_data=[]
    
    for tweet in test:
    #    tokens=text.split()
        #tokens=nltk.word_tokenize(tweet)
        tokens=tknzr.tokenize(tweet)

        if tokens !="" and tokens!=' ' and  tokens!=[]:   
            a1=[]
            for token in tokens[0:max_tokens]:
                a1.append( ft.get_word_vector(token) )
            
            a1=np.asarray(a1, dtype=np.float32)            
            temp=np.zeros([max_tokens,embedding_dim])

            if len(tokens)>max_tokens:
                temp=a1[0:max_tokens]
            elif len(tokens)==max_tokens:
                temp=a1
            else: #if len(tokens)<max_tokens:
                temp[0:len(tokens)]=a1
        
            #x_test=temp.reshape((1,max_tokens,embedding_dim))  
            #print(np.shape(temp))
            #x_test = tf.cast(temp, tf.float32)
            test_data.append(temp)
            
        else:
            test_data.append(np.zeros([max_tokens,embedding_dim]))
              
#    test_data = tf.cast(test_data, tf.float32)
    test_data = np.asarray(test_data, dtype=np.float32)
    res_test=classifier.predict(test_data)
    results_ent_test_new [ batch: batch +len (res_test)]= res_test
    batch += batch_size
     
time.time()-t0

5459.147403001785

In [80]:
with open("./years/"+year+"result_ent_bin_3.txt", "wb") as fp:  
    pickle.dump(results_ent_test_new,fp)


In [1]:
'''
n_correct=0
false_values=[]
correct_values=[]

for k,i in enumerate(results_ent_test_new):
    #print(k)
    #print(i)
    if i>0.5:
        temp=1
    else:
        temp=-1
    
    #print(k)
    #temp1=k+len(seed_public)
    #print(temp1)
    #print(id_ent_rel[temp1])
    #print(dic_ent_tweets_scores[id_ent_rel[temp1]])
    #print(dic_ent_tweets_scores[id_ent_rel[len(seed_public)+k]])
    if np.sign(temp) == np.sign(dic_ent_tweets_scores[id_new[k]]):
        n_correct +=1
        #correct_values.append(dic_ent_tweets_scores[id_ent_rel[len(seed_negative)+k]])
        #correct_values.append(dic_ent_tweets_scores[id_ent_rel[k]])
        correct_values.append(dic_ent_tweets_scores[id_new[k]])

    else:
        #false_values.append(dic_ent_tweets_scores[id_ent_rel[len(seed_negative)+k]])
        #false_values.append(dic_ent_tweets_scores[id_ent_rel[k]])
        false_values.append(dic_ent_tweets_scores[id_new[k]])


        
accuracy=    n_correct/len(results_ent_test_new)
accuracy
'''
pass

In [62]:
#np.min(correct_values), np.min(false_values)

In [63]:
#np.max(correct_values), np.max(false_values)

In [408]:
'''
positive_values=[]

for i in id_ent_rel:
    positive_values.append(dic_ent_tweets_scores[i])
'''
pass

In [68]:
#id_ent_rel[0],dic_ent_tweets_scores[id_ent_rel[0]]

In [69]:
#id_ent_irrel[0],dic_ent_tweets_scores[id_ent_irrel[0]]

In [64]:
'''
import numpy as np
import matplotlib.pyplot as plt

#fig = plt.figure()
fig = plt.figure(figsize=(8, 10))

plt.title('false distribution')
plt.xlabel(' score')
plt.ylabel('number of samples')
#n_max=dic_ent_tweets_scores[id_ent_rel[len(seed_public)]]
#x = np.random.randn(1000)
plt.hist(false_values, bins=100)
fig.savefig('./all_neg_first_pos/'+year+'/false_distribution2_all.jpg')
'''
pass

In [65]:
'''
import numpy as np
import matplotlib.pyplot as plt

#from matplotlib.pyplot import figure
#figure(figsize=(8, 6), dpi=80)

fig = plt.figure(figsize=(8, 10))


plt.title('correct distribution')
plt.xlabel(' score')
plt.ylabel('number of samples')
#n_max=dic_ent_tweets_scores[id_ent_rel[len(seed_public)]]
#x = np.random.randn(1000)
plt.hist(correct_values, bins=100)
fig.savefig('./all_neg_first_pos/'+year+'/correct_distribution2_all.jpg')
'''
pass

In [66]:
'''
import numpy as np
import matplotlib.pyplot as plt

#from matplotlib.pyplot import figure
#figure(figsize=(8, 6), dpi=80)

fig = plt.figure(figsize=(8, 10))
plt.title('all positive distribution')
plt.xlabel('positive score')
plt.ylabel('number of samples')
#n_max=dic_ent_tweets_scores[id_ent_rel[len(seed_public)]]
#x = np.random.randn(1000)
plt.hist(positive_values, bins=40)
fig.savefig('./all_neg_first_pos/2017/all_distribution2.jpg')
'''
pass

In [67]:
#len(false_values), len(correct_values), len(false_values)+len(correct_values)

In [ ]:
#time.sleep(10*60)

In [ ]:
#with open("ent_results300.txt", "wb") as fp:   
#    pickle.dump(results_ent, fp , protocol=4)

In [ ]:
#results_ent_numpy=np.asarray(results_ent, dtype=np.float32)

In [ ]:
#len(results_ent)

In [ ]:
#with open("ent_results300_np.txt", "wb") as fp:   
#    pickle.dump(results_ent_numpy, fp , protocol=4)